In [8]:
!pip install --upgrade tables
!pip install eli5

Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)


In [9]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [10]:
cd "/content/drive/My Drive/Colab Notebooks/car_prices/Car_price_prediction/"

/content/drive/My Drive/Colab Notebooks/car_prices/Car_price_prediction


In [11]:
!ls

2_visualisation.ipynb  3_simple_model.ipynb  data  LICENSE  README.md


In [0]:
df = pd.read_hdf('data/car.h5')

In [13]:
df.shape

(106494, 155)

In [14]:
df.columns

Index(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki-parkowania-przednie',
       'feature_poduszka-powietrzna-chroniąca-kolana',
       ...
       'param_pearl', 'param_stan', 'param_wersja', 'param_emisja-co2',
       'param_body-type', 'param_matowy', 'param_bezwypadkowy',
       'param_akryl-(niemetalizowany)', 'param_monthly-payment-value',
       'car_id'],
      dtype='object', length=155)

In [0]:
# Optimalization, reducing those 155 columns.

---
## Dummy Model

In [16]:
# Which columns are int or flow?
df.select_dtypes(np.number).columns

Index(['price_value', 'car_id'], dtype='object')

In [0]:
feats = ['car_id']
X = df[feats].values
y = df['price_value'].values # only vector

In [18]:
model = DummyRegressor()

model.fit(X,y)
y_pred = model.predict(X)

mae(y,y_pred) # our error (mean and absolute) is 39465

39465.934630440985

In [19]:
# Which columns have price in their name?

[x for x in df.columns if 'price' in x]

['price_currency', 'price_details', 'price_value']

In [20]:
# price_currency

df['price_currency'].value_counts()

PLN    106290
EUR       204
Name: price_currency, dtype: int64

In [21]:
df['price_currency'].value_counts(normalize=True) * 100 # getting the % partial values

PLN    99.80844
EUR     0.19156
Name: price_currency, dtype: float64

In [22]:
# we got pln and eur we could multiply eur to get pln value or remove eur

# removing eur (usually its better to convert eru to pln)

df = df[df['price_currency'] != 'EUR']
df['price_currency'].value_counts()

PLN    106290
Name: price_currency, dtype: int64

---
## Features

Feature engineering.

In [23]:
df['param_color'].factorize()[0]

array([-1, -1, -1, ..., -1, -1, -1])

In [0]:
# Quick data engineering.
# Factorizing all the columns (that are not lists). New factorized columns with suffix '__cat'.

SUFFIX_CAT = '__cat'

for feat in df.columns:
  # TypeError: unhashable type:'list' - checking if its list, y = continue.
  if isinstance(df[feat][0],list):
    continue
  # To avoid doubling adding __cat__cat, if we already factorized the feature then we will overwrite it.
  
  factorized_values = df[feat].factorize()[0]
  
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  #If feature hasn't been factorized then factorize it.
  else:  
    df[feat + SUFFIX_CAT] = factorized_values



In [29]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
len(cat_feats)

154

In [30]:
# Removing columns with word price

cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
X = df[cat_feats].values
y = df['price_value'].values

In [0]:
model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model,X,y,cv=10, scoring='neg_mean_absolute_error')

In [40]:
# 10 scoress (cv=10)
scores

array([-19724.83386724, -19659.8666365 , -19456.41798803, -19912.25748082,
       -19417.75841258, -19949.16643744, -19970.16021452, -19974.59248268,
       -19130.89663605, -19324.16940096])

In [41]:
np.mean(scores)

-19652.01195568306

In [45]:
model_2 = DecisionTreeRegressor(max_depth=5)
model_2.fit(X,y)

imp = PermutationImportance(model_2, random_state=0).fit(X,y)
eli5.show_weights(imp, feature_names=cat_feats)

Weight,Feature
0.2533 ± 0.0040,param_napęd__cat
0.2008 ± 0.0047,param_faktura-vat__cat
0.1943 ± 0.0088,param_stan__cat
0.1423 ± 0.0086,param_rok-produkcji__cat
0.0629 ± 0.0047,param_moc__cat
0.0424 ± 0.0014,feature_kamera-cofania__cat
0.0412 ± 0.0008,param_skrzynia-biegów__cat
0.0286 ± 0.0037,param_marka-pojazdu__cat
0.0191 ± 0.0022,param_pojemność-skokowa__cat
0.0163 ± 0.0005,feature_bluetooth__cat
